In [1]:
# Google Colab required
import math
import sys
import os
import time
os.environ["OMP_NUM_THREADS"] = "5" # export OMP_NUM_THREADS=5
os.environ["OPENBLAS_NUM_THREADS"] = "5" # export OPENBLAS_NUM_THREADS=5
os.environ["MKL_NUM_THREADS"] = "5" # export MKL_NUM_THREADS=5
os.environ["VECLIB_MAXIMUM_THREADS"] = "5" # export VECLIB_MAXIMUM_THREADS=5
os.environ["NUMEXPR_NUM_THREADS"] = "5" # export NUMEXPR_NUM_THREADS=5
import torch
from torch.autograd import Variable
from torch.nn import functional as F
import torch.nn as nn
import torchvision.datasets as datasets
from scipy.integrate import odeint, solve_ivp
from scipy.linalg import expm, qr
import scipy.io as sio
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import log_loss
import copy
from tqdm import tqdm
import pandas as pd
device = torch.device('cpu')
import copy
from fastprogress.fastprogress import master_bar, progress_bar
import random
import urllib.request
urllib.request.urlretrieve('https://github.com/MerkulovDaniil/split-sgd/blob/master/Code/lls_data/fanlinear.mat', 'fanlinear.mat')
urllib.request.urlretrieve('https://github.com/MerkulovDaniil/split-sgd/blob/master/Code/lls_data/shepplogan.mat', 'shepplogan.mat')
# Reproducibility
random.seed(999)
np.random.seed(999)
torch.manual_seed(999)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
print('Libraries imported successfuly!')

Libraries imported successfuly!


# Linear 

In [31]:
TARGET_RES = 16e-5 # desired loss rate
N_EXPERIMENTS = 11
LEARNING_RATES = np.array(np.logspace(-0.2, 0.8, 15))
iter_limit = 10000 # limit to iterations due to not converging
GD_N_iter = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SGD_N_iter = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SPL_N_iter = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
GD_time = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SGD_time = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SPL_time = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
p = 100 
n = 1000 # random data array [n x p]
s = 50 # split count
b = 20 # batch size
epsilon = 1e-2 # noise rate, min noise converge early
problems = [
            #'tomography',
            'random'
            ] # dataset types
print('Problem parameters defined successfuly!')

Problem parameters defined successfuly!


In [33]:
# Load Tom LLS data
def loadTomData():
    X = sio.loadmat(os.path.join(os.path.abspath(os.curdir) ,'fanlinear.mat'))["A"].toarray()
    theta_true = sio.loadmat(os.path.join(os.path.abspath(os.curdir) ,'shepplogan.mat'))["x"]
    y = np.squeeze(X @ theta_true)
    return X, theta_true, y

# Creating random X [n x p]
# Formula definition
# Create y with noise
def generateRandomProblem(p, n, lstsq=False, eps = 0):
    X = np.random.randn(n, p)
    theta_clean = np.ones(p)
    y = X @ theta_clean + eps * np.random.randn(n) 
    init_bound = 1.0/math.sqrt(p)
    theta_0 = np.array(init_bound*torch.FloatTensor(p).uniform_(-1, 1))
    if lstsq == False:
        return X, theta_0, y
    else:
        theta_lstsq = np.linalg.lstsq(X,y)[0]
        return X, theta_0, y, theta_lstsq

# C.1.2 EXACT SOLUTION OF THE LOCAL PROBLEM
def exactSolution(Q, R, theta_0, y_batch, h, n):
    try:
        R_it = np.linalg.inv(R.T)
    except:
        R_it = np.linalg.pinv(R.T)
    return Q @ ( expm(-1/n* R @ R.T*h) @ (Q.T @ theta_0 - R_it @ y_batch )) + Q @ (R_it @ y_batch) + theta_0 - Q @ (Q.T @ theta_0)

# C.1.3 formula 21 - exact formula for Kaczmarz method
def KaczmarzMethod(x, theta_0, y, h, n):
    x = x.T
    norm = x.T @ x
    return theta_0 + (1 - np.exp(-norm*h/n))*(y - x.T @ theta_0)/norm*x

# Loss function of Linear Least Sq
def loss(X, theta, y):
    if len(X.shape) == 2:
        n, _ = X.shape
        return 1/n*np.linalg.norm(X @ theta - y)**2
    elif len(X.shape) == 3:
        s, b, _ = Xs.shape
        n = b * s
        loss = 0
        for i in range(s):
            loss += 1/n*np.linalg.norm(X[i] @ theta - y[i])**2
        return loss
    else:
        print('HITTITITIITITITIIT')
        raise ValueError('Data Shape is not Suitable!!!')

# || calculated - exact || / || exact || 
# True values to compare with Loss
# Calculated for all batches at once
def exactValues(X, theta, y):
    if len(X.shape) == 2:
        return np.linalg.norm(X @ theta - y)/np.linalg.norm(y)
    elif len(X.shape) == 3:
        s, b, p = Xs.shape
        n = b * s
        loss = 0
        X_full = np.zeros((n, p))
        y_full = np.zeros(n)
        for i in range(s):
            y_full[b*i:b*(i+1)] = y[i]
            X_full[b*i:b*(i+1),:] = X[i]
        return np.linalg.norm(X_full @ theta - y_full)/np.linalg.norm(y_full)
    else:
        print('HITTITITIITITITIIT')
        raise ValueError('Data Shape is not Suitable!!!')

# C.1.1 formula 17 gradient update
def calculateGradient(X, theta, y):
    n, p = X.shape
    return (1/n * (X.T @ (X @ theta - y)))

# SGD iteration step
def SGDIteration(X_batch, theta_0, y_batch, lr):
    theta = theta_0 - lr * calculateGradient(X_batch, theta_0, y_batch)
    return theta

# train SGD 
def trainSGD(thetaTemp, Xs, ys, lr, treshold = 1e-4, maxIter = 1000):
    s, b, _ = Xs.shape
    n = b * s
    iterCount = 0
    stopFlag = False
    losses = []
    if lr > 9:
        maxIter = 10000
    while not stopFlag:          
        i_batch = iterCount % s
        losses.append(loss(Xs, thetaTemp, ys))
        thetaTemp = SGDIteration(Xs[i_batch], thetaTemp, ys[i_batch], lr)
        iterCount += 1
        if iterCount % 50 == 0:
            print(f'SGD exactValues {exactValues(Xs, thetaTemp, ys):.5f}, losses {losses[-1]:.3f}/{treshold:.5f}, iteration {iterCount}. Lr {lr}')
        if losses[-1] <= treshold:
            stopFlag = True
            break
        if iterCount >= maxIter:
            stopFlag = True
            iterCount = None
    if iterCount == None:
      print(f'\n SGD complete with fail. iteration {iterCount}, lr {lr}')
    else:
      print(f'\n SGD complete with success. iteration {iterCount}, lr {lr}')
    return iterCount

# train splitting 
def trainSplitting(thetaTemp, Qs, Rs, Xs, ys, stepsize, treshold = 1e-4, maxIter = 1000):
    s, b, _ = Xs.shape
    n = b * s
    iterCount = 0
    stopFlag = False
    losses = []
    while not stopFlag:          
        i_batch = iterCount % s
        losses.append(loss(Xs, thetaTemp, ys))
        thetaTemp = exactSolution(Qs[i_batch], Rs[i_batch], thetaTemp, ys[i_batch], stepsize, n)
        iterCount += 1
        if iterCount % 50 == 0:
            print(f'Splitting exactValues {exactValues(Xs, thetaTemp, ys)}, losses {losses[-1]:.5f}/{treshold:.5f}, iteration {iterCount}. Stepsize {stepsize}')
        if losses[-1] <= treshold:
            stopFlag = True
            break
        if losses[-1] >= 1e4 or iterCount >= maxIter:
            stopFlag = True
            iterCount = None
    if iterCount == None:
      print(f'\n Splitting complete with fail. iteration {iterCount}, Stepsize {stepsize}')
    else:
      print(f'\n Splitting complete with success. iteration {iterCount}, Stepsize {stepsize}')
    return iterCount

# Drawings helper
def drawLearningRate2Time(learning_rates, list_of_methods, list_of_labels):
    colors = ['g', 'r']
    color_labels = ['^', 'o']
    plt.figure(figsize = (3.5,2.5))
    for method, label, color, col_lab in zip(list_of_methods, list_of_labels, colors, color_labels):
        mean = np.zeros(len(learning_rates))
        std = np.zeros(len(learning_rates))
        for i_lr, lr in enumerate(learning_rates):
            if any(method[:, i_lr]) == None:
                mean[i_lr] = None
                std[i_lr] = None
            else:
                mean[i_lr] = np.mean(method[:, i_lr])
                std[i_lr] = np.std(method[:, i_lr])
        plt.loglog(learning_rates, mean, color+col_lab, label = label)
        plt.loglog(learning_rates, mean, color+':')
        plt.fill_between(learning_rates, mean-std, mean+std, color=color, alpha=0.1)
        plt.grid(True,which="both", linestyle='--', linewidth=0.4)
        plt.xlabel('Learning rate')
        plt.ylabel('Time to converge')
        plt.legend()
    plt.tight_layout()
    plt.show()

# Drawings helper
def drawLearningRate2IterCount(learning_rates, list_of_methods, list_of_labels):
    colors = ['g', 'r']
    color_labels = ['^', 'o']
    plt.figure(figsize = (3.5,2.5))
    for method, label, color, col_lab in zip(list_of_methods, list_of_labels, colors, color_labels):
        mean    = np.zeros(len(learning_rates))
        std     = np.zeros(len(learning_rates))

        for i_lr, lr in enumerate(learning_rates):
            if any(method[:, i_lr]) == None:
                mean[i_lr] = None
                std[i_lr]  = None
            else:
                mean[i_lr] = np.mean(method[:, i_lr])
                std[i_lr]  = np.std(method[:, i_lr])
        std = np.std(method, axis = 0)   
        plt.loglog(learning_rates, mean, color+col_lab, label = label)
        plt.loglog(learning_rates, mean, color+':')
        plt.fill_between(learning_rates, mean-std, mean+std, color=color, alpha=0.1)
        plt.grid(True,which="both", linestyle='--', linewidth=0.4)
        plt.xlabel('Learning rate')
        plt.ylabel('Iterations to converge')
        plt.legend()
    plt.tight_layout()
    plt.show()

print('Functions defined successfuly!')

Functions defined successfuly!


In [0]:
for problem in problems:
    print(f'Problem {problem} ')
    if problem == 'tomography': # if data set is LLS, reassign parameters
        X, theta_true, y = loadTomData()
        n, p = X.shape
        b, s = 60, 213

    for i_exp in progress_bar(range(N_EXPERIMENTS)):
        print(f'EXPERIMENT {i_exp + 1} / {N_EXPERIMENTS} ')
        # Arranging dataset depend on problem
        if problem == 'tomography':
            init_bound = 1.0/math.sqrt(p)
            theta_0 = np.array(init_bound*torch.FloatTensor(p).uniform_(-1, 1))
            permutation = np.random.permutation(n)
            X, y = X[permutation], y[permutation]
        elif problem == 'random':
            X, theta_0, y, theta_lstsq = generateRandomProblem(p,n, lstsq=True, eps=epsilon)

        # Dividing batches
        Xs = np.zeros((s, b, p))
        ys = np.zeros((s, b))
        Qs = np.zeros((s, p, b))
        Rs = np.zeros((s, b, b))
        Q, R = qr(X.T, mode='economic')
        for i_batch in range(s):
            Xs[i_batch] = X[b*i_batch:b*(i_batch+1), :]
            ys[i_batch] = y[b*i_batch:b*(i_batch+1)]
            Qs[i_batch], Rs[i_batch] = qr(Xs[i_batch].T, mode='economic')
            
        # Start to train and report
        for i_lr, learning_rate in enumerate(LEARNING_RATES):
            stepsize = learning_rate*n/b
            print(f'learning_rate {learning_rate}, stepsize {stepsize} ')

            start_time = time.time()
            N_iter = trainSplitting(theta_0, Qs, Rs, Xs, ys, stepsize, treshold = TARGET_RES, maxIter = iter_limit)
            end_time = time.time()
            SPL_time[i_exp, i_lr] = end_time - start_time
            SPL_N_iter[i_exp, i_lr] = N_iter
            if N_iter == None:
                SPL_N_iter[i_exp, i_lr] = None

            start_time = time.time()
            N_iter = trainSGD(theta_0, Xs, ys, learning_rate, treshold = TARGET_RES, maxIter = iter_limit)
            end_time = time.time()
            SGD_time[i_exp, i_lr] = end_time - start_time
            SGD_N_iter[i_exp, i_lr] = N_iter
            if N_iter == None:
                SGD_time[i_exp, i_lr] = None

            drawLearningRate2Time(LEARNING_RATES, [SPL_time, SGD_time], ['Splitting','SGD'])
            drawLearningRate2IterCount(LEARNING_RATES, [SPL_N_iter, SGD_N_iter], ['Splitting','SGD'])

# Logistic

In [0]:
TARGET_ERROR    = 1e-3 # desired loss rate
N_EXPERIMENTS   = 30
LEARNING_RATES  = np.array(np.logspace(-3, 2, 10))
LEARNING_RATES  = np.append(LEARNING_RATES, np.array(np.logspace(2, 7, 6)))
iter_limit      = 30000 # limit to iterations due to not converging
GD_N_iter     = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SGD_N_iter    = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SPL_N_iter    = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
GD_time     = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SGD_time    = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SPL_time    = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
batch_size = 50 # batch size
number_of_classes = 2

print('Problem parameters defined successfuly!')

In [0]:
# Calculates sigmoid funcion = 1/(1 + exp(-x_i))
def sigmoid(x):
    if np.isscalar(x):
        return 1/(1 + np.exp(-x))
    else:
        return np.array([1/(1 + np.exp(-x_i)) for x_i in x])

# Load data from CSV
# Apply pruning to reshape
# Create train and test set
# Creating batches from train and test sets
def load_batched_data_epi(batch_size=50, shuffle = True, qr_mode = False, number_of_classes = 2):
    data = pd.read_csv('logreg/data.csv')
    print(f'Before pruning {data.shape}')
    data = data.dropna()
    y = data['y']
    X = data.drop(data.columns[0], axis=1)
    X = X.drop(columns=['y'])
    select_binary = (y == 2) + (y == 1)
    X, y = X[select_binary], y[select_binary]
    print(f'After pruning {data.shape}, {X.shape}, {y.shape}')
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    n_train, p = X_train.shape
    n_test = len(y_test)
    s_train = int(n_train/batch_size)   
    K = number_of_classes 
    X_trains = torch.zeros((s_train, batch_size, p), requires_grad=False).to(device)
    y_trains = torch.zeros((s_train, batch_size), requires_grad=False).to(device)
    if qr_mode:
        Qs = torch.zeros((s_train, p, batch_size), requires_grad=False).to(device)
        Rs = torch.zeros((s_train, batch_size, batch_size), requires_grad=False).to(device)
    for i in range(s_train):
        X_trains[i] = torch.from_numpy(X_train[batch_size*i:batch_size*(i+1)].to_numpy())
        y_trains[i] = torch.from_numpy(y_train[batch_size*i:batch_size*(i+1)].to_numpy())
        if qr_mode:
            Qs[i], Rs[i] = torch.qr(X_trains[i].t())      
    print(type(X_trains), type(y_trains), type(X_test), type(y_test), type(Qs), type(Rs))        
    if qr_mode:
        return X_trains, y_trains, torch.from_numpy(X_test.to_numpy()), torch.from_numpy(y_test.to_numpy()), Qs, Rs
    else:
        return X_trains, y_trains, torch.from_numpy(X_test.to_numpy()), torch.from_numpy(y_test.to_numpy())


# Load data from MNIST
# Create train and test set
# Apply shuffling to handle memorizing
# Creating batches from train and test sets
def load_batched_data(batch_size=50, shuffle = True, qr_mode = False, number_of_classes = 2):
    trainset = datasets.MNIST('./mnist_data/', download=True, train=True)
    X_train = trainset.train_data.to(dtype=torch.float)/255
    y_train = trainset.train_labels
    mask = y_train < number_of_classes
    X_train = X_train[mask]
    y_train = y_train[mask]
    X_train.resize_(len(X_train), *X_train[0].view(-1).shape)
    y_train.view(-1).long()
    testset = datasets.MNIST('./mnist_data/', download=True, train=False)
    X_test = testset.test_data.to(dtype=torch.float)/255
    y_test = testset.test_labels
    mask = y_test < number_of_classes
    X_test = X_test[mask]
    y_test = y_test[mask]
    X_test.resize_(len(X_test), *X_test[0].view(-1).shape)
    y_test.view(-1).long()
    if shuffle == True:
        shuffling = torch.randperm(len(y_train))
        X_train = X_train[shuffling]
        y_train = y_train[shuffling]
    if shuffle == True:
        shuffling = torch.randperm(len(y_test))
        X_test = X_test[shuffling].to(device)
        y_test = y_test[shuffling]
    n_train = len(y_train)
    n_test  = len(y_test)
    s_train = int(n_train/batch_size)  
    K = number_of_classes 
    X_trains = torch.zeros((s_train, batch_size, *X_train[0].view(-1).shape), requires_grad=False).to(device)
    y_trains = torch.zeros((s_train, batch_size), requires_grad=False).to(device)
    if qr_mode:
        Qs = torch.zeros((s_train, *X_train[0].view(-1).shape, batch_size), requires_grad=False).to(device)
        Rs = torch.zeros((s_train, batch_size, batch_size), requires_grad=False).to(device)
    for i in range(s_train):
        X_trains[i] = X_train[batch_size*i:batch_size*(i+1), :]
        y_trains[i] = y_train[batch_size*i:batch_size*(i+1)]
        if qr_mode:
            Qs[i], Rs[i] = torch.qr(X_trains[i].t())      
    if qr_mode:
        return X_trains, y_trains, X_test, y_test, Qs, Rs
    else:
        return X_trains, y_trains, X_test, y_test

# Creating Logistic Regression Model
# Adding one Linear Layer
# using sigmoid activation function to make prediction
class LogisticRegression(torch.nn.Module):
     def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(p, 1)
     def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

# Merge dataset batches to single array
def full_problem_from_batches(Xs, ys):
    s_train, batch_size, p = Xs.shape
    X = torch.zeros(s_train*batch_size, p)
    y = torch.zeros(s_train*batch_size)
    for i_batch in range(s_train):
        X[batch_size*i_batch:batch_size*(i_batch+1), :] = Xs[i_batch]
        y[batch_size*i_batch:batch_size*(i_batch+1)]    = ys[i_batch]
    return X, y

# Creating new model instance
# We won't update bias during the training, since they are not affect the model predictions
def model_init(model, parameters_tensor):
    new_model = copy.deepcopy(model)
    for parameter in new_model.parameters():
        parameter.data = parameters_tensor.clone().to(device)
        break
    return new_model

# train SGD 
def sgd_training(theta_0, X_trains, y_trains,  X_test, y_test, lr, model, final_error = 0.2, iter_limit = 1000):
    X, y = full_problem_from_batches(X_trains, y_trains)
    X, y, X_test, y_test = X.float().to(device), y.float().to(device), X_test.to(device), y_test.to(device)
    model = model.to(device)
    s_train, batch_size, p = X_trains.shape
    n_train, p  = X.shape
    n_test = len(y_test)
    thetas = []
    losses_train = []
    errors_train = []
    losses_test = []
    errors_test = []
    criterion = torch.nn.BCELoss()
    theta_t = theta_0
    model = model_init(model, theta_0.t())
    stop_word = False
    N_iter = 0
    if lr >= 0.2:
        iter_limit = 1000
    while not stop_word:          
        i_batch = N_iter % s_train
        if i_batch % 1 == 0:
            model.eval()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            thetas.append(theta_t)
            losses_train.append(loss.data)
            pred_labels     = torch.squeeze(y_pred >= 0.5).float()
            train_acc       = y.eq(pred_labels.data).sum().to(dtype=torch.float)/len(pred_labels)
            errors_train.append(1 - train_acc) 
            y_pred_test = model(X_test)
            loss_test   = criterion(y_pred_test, y_test.float())
            losses_test.append(loss_test.data)
            pred_labels_test    = torch.squeeze(y_pred_test >= 0.5).long()
            test_acc            = y_test.eq(pred_labels_test.data).sum().to(dtype=torch.float)/len(y_pred_test)
            errors_test.append(1 - test_acc)
            if errors_test[-1] <= final_error:
                stop_word = True
                break
            if N_iter >= iter_limit:
                N_iter = None
                print(f'\n🤖 SGD Failed on lr {lr}')
                return N_iter, thetas, losses_train,losses_test, errors_train, errors_test
        model.train()
        model.zero_grad()
        y_pred = model(X_trains[i_batch])
        loss = criterion(y_pred, y_trains[i_batch])
        loss.backward()
        for parameter in model.parameters():
            parameter.data = parameter.data - lr*parameter.grad.data
            theta_t = np.array((parameter.data.t()).cpu())
            break
        N_iter += 1
    print(f'SGD finished with {N_iter} iterations on lr {lr}')
    return N_iter, thetas, losses_train,losses_test, errors_train, errors_test

# C.2.2 function 27 
# SPLITTING SCHEME
# integrate from 0 to h
def make_splitting_step(theta_0, Q, R, y, h, n):
    h_seq = [0, h]
    Q, R, theta_0 = np.array(Q), np.array(R), np.array(theta_0)
    eta_0, theta_0 = np.squeeze(Q.T@theta_0), np.squeeze(theta_0)
    def rhs(eta, t):
        return -1/n * R@(sigmoid(R.T @ eta) - np.array(y))
    eta_h = odeint(rhs, eta_0, h_seq)[-1]
    theta = Q@(eta_h - eta_0) + theta_0
    return torch.from_numpy(theta).reshape(p, 1)

# train splitting 
def spl_training(theta_0, Qs, Rs, X_trains, y_trains,  X_test, y_test, stepsize, model, final_error = 0.2, iter_limit = 1000):
    X, y = full_problem_from_batches(X_trains, y_trains)
    X, y, X_trains, y_trains, X_test, y_test, model = X.float().to(device), y.float().to(device), X_trains.float().to(device), y_trains.float().to(device), X_test.float().to(device), y_test.float().to(device), model.to(device)
    s_train, batch_size, p = X_trains.shape
    n_train, p  = X.shape
    n_test = len(y_test)
    thetas = []
    losses_train = []
    errors_train = []
    losses_test = []
    errors_test = []
    criterion = torch.nn.BCELoss()
    theta_t = theta_0.to(device)
    model = model_init(model, theta_0.t())
    stop_word = False
    N_iter = 0
    if stepsize >= 1000:
        iter_limit = 1000
    while not stop_word:
        i_batch = N_iter % s_train
        if i_batch % 1 == 0:      
            model.eval()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            thetas.append(theta_t)
            losses_train.append(loss.data)
            pred_labels = torch.squeeze(y_pred >= 0.5).float()
            train_acc = y.eq(pred_labels.data).sum().to(dtype=torch.float)/len(pred_labels)
            errors_train.append(1 - train_acc) 
            y_pred_test = model(X_test)
            loss_test = criterion(y_pred_test, y_test)
            losses_test.append(loss_test.data)
            pred_labels_test = torch.squeeze(y_pred_test >= 0.5).float()
            test_acc = y_test.eq(pred_labels_test.data).sum().to(dtype=torch.float)/len(y_pred_test)
            errors_test.append(1 - test_acc)
            if errors_test[-1] <= final_error:
                stop_word = True
                break
            if N_iter >= iter_limit:
                N_iter = None
                print(f'\n Splitting Failed on lr {lr}')
                return N_iter, thetas, losses_train,losses_test, errors_train, errors_test
        model.train()
        theta_t = make_splitting_step(theta_t.cpu(), Qs[i_batch].cpu(), Rs[i_batch].cpu(), y_trains[i_batch].cpu(), stepsize, n_train).to(dtype=torch.float)
        model = model_init(model, theta_t.t())
        N_iter += 1  
    print(f' Splitting finished with {N_iter} iterations on Stepsize {stepsize}')
    return N_iter, thetas, losses_train,losses_test, errors_train, errors_test

# Drawings helper
def drawLearningRateTime(learning_rates, list_of_methods, list_of_labels):
    colors = ['g', 'r']
    color_labels = ['^', 'o']
    plt.figure(figsize = (3.5,2.5))
    for method, label, color, col_lab in zip(list_of_methods, list_of_labels, colors, color_labels):
        mean = np.zeros(len(learning_rates))
        std = np.zeros(len(learning_rates))
        for i_lr, lr in enumerate(learning_rates):
            if any(method[:, i_lr]) == None:
                mean[i_lr] = None
                std[i_lr]  = None
            else:
                mean[i_lr] = np.mean(method[:, i_lr])
                std[i_lr]  = np.std(method[:, i_lr])
        plt.loglog(learning_rates, mean, color+col_lab, label = label)
        plt.loglog(learning_rates, mean, color+':')
        plt.fill_between(learning_rates, mean-std, mean+std, color=color, alpha=0.1)
        plt.grid(True,which="both", linestyle='--', linewidth=0.4)
        plt.xlabel('Learning rate')
        plt.ylabel('Time to converge')
        plt.legend()
    plt.tight_layout()
    plt.show()

# Drawings helper
def drawLearningRateIterCount(learning_rates, list_of_methods, list_of_labels):
    colors = ['g', 'r']
    color_labels = ['^', 'o']
    plt.figure(figsize = (3.5,2.5))
    for method, label, color, col_lab in zip(list_of_methods, list_of_labels, colors, color_labels):
        mean = np.zeros(len(learning_rates))
        std = np.zeros(len(learning_rates))
        for i_lr, lr in enumerate(learning_rates):
            if any(method[:, i_lr]) == None:
                mean[i_lr] = None
                std[i_lr]  = None
            else:
                mean[i_lr] = np.mean(method[:, i_lr])
                std[i_lr]  = np.std(method[:, i_lr])
        std = np.std(method, axis = 0)   
        plt.loglog(learning_rates, mean, color+col_lab, label = label)
        plt.loglog(learning_rates, mean, color+':')
        plt.fill_between(learning_rates, mean-std, mean+std, color=color, alpha=0.1)
        plt.grid(True,which="both", linestyle='--', linewidth=0.4)
        plt.xlabel('Learning rate')
        plt.ylabel('Iterations to converge')
        plt.legend()
    plt.tight_layout()
    plt.show()

print('Functions defined Successfuly!!')

In [0]:
# Assigning dataset, batches and start to train
X_trains, y_trains, X_test, y_test, Qs, Rs = load_batched_data_epi(batch_size=batch_size, qr_mode = True, number_of_classes=number_of_classes)
s_train, batch_size, p = X_trains.shape 
n_train, n_test = s_train*batch_size, len(y_test)
model = LogisticRegression()
for i_exp in progress_bar(range(N_EXPERIMENTS)):
    print(f'EXPERIMENT {i_exp+1}/ {N_EXPERIMENTS} ')
    init_bound = 1.0/math.sqrt(p)
    theta_0 = init_bound*torch.FloatTensor(p, 1).uniform_(-1, 1)
    for i_lr, learning_rate in enumerate(LEARNING_RATES):
        stepsize = learning_rate*n_train/batch_size
        print(f'learning_rate {learning_rate}, stepsize {stepsize} ')
        start_time = time.time()
        N_iter, thetas, losses_train,losses_test, errors_train, errors_test = \
            spl_training(theta_0,  Qs, Rs, X_trains, y_trains,  X_test, y_test, stepsize, model, final_error = TARGET_ERROR, iter_limit=iter_limit)
        end_time = time.time()
        SPL_time[i_exp, i_lr] = end_time - start_time
        SPL_N_iter[i_exp, i_lr] = N_iter
        if N_iter == None:
            SPL_N_iter[i_exp, i_lr] = None
        start_time = time.time()
        N_iter, thetas, losses_train,losses_test, errors_train, errors_test = \
            sgd_training(theta_0, X_trains, y_trains,  X_test, y_test, learning_rate, model, final_error = TARGET_ERROR, iter_limit=iter_limit)
        end_time = time.time()
        SGD_time[i_exp, i_lr] = end_time - start_time
        SGD_N_iter[i_exp, i_lr] = N_iter
        if N_iter == None:
            SGD_time[i_exp, i_lr] = None

        drawLearningRateTime(LEARNING_RATES, [SPL_time, SGD_time], ['Splitting','SGD'])
        drawLearningRateIterCount(LEARNING_RATES, [SPL_N_iter, SGD_N_iter], ['Splitting','SGD'])

# Softmax

In [0]:
TARGET_ERROR = 0.25 # desired loss rate
N_EXPERIMENTS = 10
LEARNING_RATES = np.array(np.logspace(-2, 1.5, 10))[::-1]
iter_limit = 300000#  limit to iterations due to not converging
GD_time = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SGD_time = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SPL_time = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
GD_N_iter = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SGD_N_iter = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
SPL_N_iter = np.zeros((N_EXPERIMENTS, len(LEARNING_RATES)))
batch_size = 64 # batch size
number_of_classes = 10

print('Problem parameters defined successfuly!')

In [0]:
# Load data from MNIST
# Create train and test set
# Apply shuffling to handle memorizing
# Creating batches from train and test sets
def load_batched_data(batch_size=50, shuffle = True, qr_mode = False, number_of_classes = 3):
    trainset = datasets.FashionMNIST('./fashion_mnist_data/', download=True, train=True)
    X_train = trainset.train_data.to(dtype=torch.float)/255
    y_train = trainset.train_labels
    mask = y_train < number_of_classes
    X_train = X_train[mask]
    y_train = y_train[mask]
    X_train.resize_(len(X_train), *X_train[0].view(-1).shape)
    y_train.view(-1).long()
    testset = datasets.FashionMNIST('./fashion_mnist_data/', download=True, train=False)
    X_test = testset.test_data.to(dtype=torch.float)/255
    y_test = testset.test_labels
    mask = y_test < number_of_classes
    X_test = X_test[mask]
    y_test = y_test[mask]
    X_test.resize_(len(X_test), *X_test[0].view(-1).shape)
    y_test.view(-1).long()
    if shuffle == True:
        shuffling = torch.randperm(len(y_train))
        X_train = X_train[shuffling]
        y_train = y_train[shuffling]
    if shuffle == True:
        shuffling = torch.randperm(len(y_test))
        X_test = X_test[shuffling].to(device)
        y_test = y_test[shuffling]
    n_train = len(y_train)
    n_test = len(y_test)
    s_train = int(n_train/batch_size)   
    K = number_of_classes 
    X_trains = torch.zeros((s_train, batch_size, *X_train[0].view(-1).shape), requires_grad=False).to(device)
    y_trains = torch.zeros((s_train, K, batch_size), requires_grad=False, dtype=torch.int64).to(device)
    if qr_mode:
        Qs = torch.zeros((s_train, *X_train[0].view(-1).shape, batch_size), requires_grad=False).to(device)
        Rs = torch.zeros((s_train, batch_size, batch_size), requires_grad=False).to(device)
    y_test_one_hot = torch.zeros((n_test, K))
    y_test_one_hot[np.arange(n_test), y_test] = 1
    y_test_one_hot = y_test_one_hot.t()
    for i in range(s_train):
        X_trains[i] = X_train[batch_size*i:batch_size*(i+1), :]
        batch_lbls  = y_train[batch_size*i:batch_size*(i+1)]
        y_batch_one_hot = torch.zeros((batch_size, K))
        y_batch_one_hot[np.arange(batch_size), batch_lbls] = 1
        y_trains[i] = y_batch_one_hot.t()
        if qr_mode:
            Qs[i], Rs[i] = torch.qr(X_trains[i].t())      
    if qr_mode:
        return X_trains, y_trains, X_test, y_test_one_hot, Qs, Rs
    else:
        return X_trains, y_trains, X_test, y_test_one_hot

# Load data from CIFAR
# Create train and test set
# Apply shuffling to handle memorizing
# Creating batches from train and test sets
def load_batched_data_cifar(batch_size=50, shuffle = True, qr_mode = False, number_of_classes = 3):
    trainset = datasets.CIFAR10('./cifar_data/', download=True, train=True)
    X_train = torch.from_numpy(trainset.train_data).to(dtype=torch.float)/255
    y_train = torch.Tensor(trainset.train_labels)
    mask = y_train < number_of_classes
    X_train = X_train[mask]
    y_train = y_train[mask]
    X_train.resize_(len(X_train), *X_train[0].view(-1).shape)
    y_train.view(-1).long()
    testset = datasets.CIFAR10('./cifar_data/', download=True, train=False)
    X_test = torch.from_numpy(testset.test_data).to(dtype=torch.float)/255
    y_test = torch.Tensor(testset.test_labels)
    mask = y_test < number_of_classes
    X_test = X_test[mask]
    y_test = y_test[mask]
    X_test.resize_(len(X_test), *X_test[0].view(-1).shape)
    y_test.view(-1).long()
    if shuffle == True:
        shuffling = torch.randperm(len(y_train))
        X_train = X_train[shuffling]
        y_train = y_train[shuffling]
    if shuffle == True:
        shuffling = torch.randperm(len(y_test))
        X_test = X_test[shuffling].to(device)
        y_test = y_test[shuffling]
    n_train = len(y_train)
    n_test = len(y_test)
    s_train = int(n_train/batch_size) 
    K = number_of_classes 
    X_trains = torch.zeros((s_train, batch_size, *X_train[0].view(-1).shape), requires_grad=False).to(device)
    y_trains = torch.zeros((s_train, K, batch_size), requires_grad=False, dtype=torch.int64).to(device)
    if qr_mode:
        Qs = torch.zeros((s_train, *X_train[0].view(-1).shape, batch_size), requires_grad=False).to(device)
        Rs = torch.zeros((s_train, batch_size, batch_size), requires_grad=False).to(device)
    y_test_one_hot = torch.zeros((n_test, K))
    y_test_one_hot[np.arange(n_test), y_test.long()] = 1
    y_test_one_hot = y_test_one_hot.t()
    for i in range(s_train):
        X_trains[i] = X_train[batch_size*i:batch_size*(i+1), :]
        batch_lbls = y_train[batch_size*i:batch_size*(i+1)]
        y_batch_one_hot = torch.zeros((batch_size, K))
        y_batch_one_hot[np.arange(batch_size), batch_lbls.long()] = 1
        y_trains[i] = y_batch_one_hot.t()
        if qr_mode:
            Qs[i], Rs[i] = torch.qr(X_trains[i].t())      
    if qr_mode:
        return X_trains, y_trains, X_test, y_test_one_hot, Qs, Rs
    else:
        return X_trains, y_trains, X_test, y_test_one_hot

# C.3.1 function 31 and 32 softmax function 
def softmax_numpy(X):
    return np.array([np.exp(x)/sum(np.exp(x)) for x in X.T]).T

# Creating CrossEntropyLoss model
# Modify to compute one-hot encoding
class CrossEntropyLoss_one_hot(nn.CrossEntropyLoss):
    def forward(self, input, target):
        target = torch.squeeze(torch.max(target, 1, keepdim=True)[1])
        return F.cross_entropy(input, target, weight=self.weight,
                                ignore_index=self.ignore_index, reduction=self.reduction)

# Merge dataset batches to single array
def full_problem_from_batches(Xs, ys):
    s_train, batch_size, p = Xs.shape
    s_train, K, batch_size = ys.shape
    X = torch.zeros(s_train*batch_size, p)
    y = torch.zeros(K, s_train*batch_size)
    for i_batch in range(s_train):
        X[batch_size*i_batch:batch_size*(i_batch+1), :] = Xs[i_batch]
        y[:, batch_size*i_batch:batch_size*(i_batch+1)] = ys[i_batch]
    return X, y

# Creating new model instance
# We won't update bias during the training, since they are not affect the model predictions
def model_init(model, parameters_tensor):
    new_model = copy.deepcopy(model)
    for parameter in new_model.parameters():
        parameter.data = parameters_tensor.clone().to(device)
        break
    return new_model

# Creating Logistic Regression Model
# Adding one Linear Layer
# using softmax activation function to make prediction
class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(int(input_dim), int(output_dim))
    def forward(self, x):
        x = x.contiguous().view(x.size(0), -1)
        return F.softmax(self.linear(x), dim=1)

# Creating new LogisticRegression instance
def load_model(X_test, y_test):
    input_dim = X_test[0].numel()
    K, n_test = y_test.shape
    model = LogisticRegression(input_dim, K)
    return model

# train SGD
def sgd_training(theta_0, X_trains, y_trains,  X_test, y_test, lr, model, final_error = 0.2, iter_limit = 1000):
    X, y = full_problem_from_batches(X_trains, y_trains)
    X, y, X_test, y_test = X.float().to(device), y.float().to(device), X_test.to(device), y_test.to(device)
    model = model.to(device)
    s_train, batch_size, p = X_trains.shape
    n_train, p  = X.shape
    K, n_test = y_test.shape
    thetas = []
    losses_train = []
    errors_train = []
    losses_test = []
    errors_test = []
    criterion = CrossEntropyLoss_one_hot()
    theta_t = theta_0
    model = model_init(model, theta_0.t())
    stop_word = False
    N_iter = 0
    if lr >= 1:
        iter_limit = 30000
    if lr >= 10:
        iter_limit = 1000
    while not stop_word:          
        i_batch = N_iter % s_train
        if i_batch % 1 == 0:
            model.eval()
            y_pred = model(X)
            loss = criterion(y_pred, y.t())
            thetas.append(theta_t)
            losses_train.append(loss.data)
            pred_labels = torch.max(y_pred, 1, keepdim=True)[1]
            true_labels = torch.max(y.t(), 1, keepdim=True)[1]
            train_acc = true_labels.eq(pred_labels.data).sum().to(dtype=torch.float)/len(true_labels)
            errors_train.append(1 - train_acc) 
            y_pred_test = model(X_test)
            loss_test = criterion(y_pred_test, y_test.t())
            losses_test.append(loss_test.data)
            pred_labels_test = torch.max(y_pred_test, 1, keepdim=True)[1]
            true_labels_test = torch.max(y_test.t(), 1, keepdim=True)[1]
            test_acc = true_labels_test.eq(pred_labels_test.data).sum().to(dtype=torch.float)/len(true_labels_test)
            errors_test.append(1 - test_acc)
            if errors_test[-1] <= final_error:
                stop_word = True
                break
            if N_iter >= iter_limit:
                N_iter = None
                return N_iter, thetas, losses_train,losses_test, errors_train, errors_test
        model.train()
        model.zero_grad()
        y_pred = model(X_trains[i_batch])
        loss = criterion(y_pred, y_trains[i_batch].t())
        loss.backward()
        for parameter in model.parameters():
            parameter.data = parameter.data - lr*parameter.grad.data
            theta_t = np.array((parameter.data.t()).cpu())
            break
        N_iter += 1
    return N_iter, thetas, losses_train,losses_test, errors_train, errors_test
# C.3.1 Final formula to calculate one step of splitting
def make_splitting_step(theta_0, Q, R, y, h, n_train):
    p, K = theta_0.shape
    p, batch_size = Q.shape
    Q, R, y = np.array(Q), np.array(R), np.array(y)
    h_seq = [0, h]
    theta_0 = np.array(theta_0)   
    H_0 = np.array(Q.T @ theta_0)
    H_0_vec = H_0.ravel('F')
    H_h = np.zeros((batch_size, K))
    def rhs_vec(H, t):
        H = H.reshape((batch_size, K), order='F')
        rhs = -1/n_train * R@(softmax_numpy(H.T@R) - y).T
        return rhs.ravel('F')
    H_h_vec = odeint(rhs_vec, H_0_vec, h_seq)[-1]
    H_h = H_h_vec.reshape((batch_size, K), order='F')
    theta = Q@(H_h - H_0) + theta_0
    return torch.from_numpy(theta)

# Train Splitting
def spl_training(theta_0, Qs, Rs, X_trains, y_trains,  X_test, y_test, stepsize, model, final_error = 0.2, iter_limit = 1000):
    X, y = full_problem_from_batches(X_trains, y_trains)
    X, y, X_trains, y_trains, X_test, y_test, model = X.float().to(device), y.float().to(device), X_trains.float().to(device), y_trains.float().to(device), X_test.float().to(device), y_test.float().to(device), model.to(device)
    s_train, batch_size, p = X_trains.shape
    n_train, p  = X.shape
    K, n_test = y_test.shape
    thetas = []
    losses_train = []
    errors_train = []
    losses_test = []
    errors_test = []
    criterion = CrossEntropyLoss_one_hot()
    theta_t = theta_0.to(device)
    model = model_init(model, theta_0.t())
    stop_word = False
    N_iter = 0
    if stepsize >= 1000:
        iter_limit = 1000
    while not stop_word:
        i_batch = N_iter % s_train
        if i_batch % 1 == 0:      
            model.eval()
            y_pred = model(X)
            loss = criterion(y_pred, y.t())
            thetas.append(theta_t)
            losses_train.append(loss.data)
            pred_labels = torch.max(y_pred, 1, keepdim=True)[1]
            true_labels = torch.max(y.t(), 1, keepdim=True)[1]
            train_acc = true_labels.eq(pred_labels.data).sum().to(dtype=torch.float)/len(true_labels)
            errors_train.append(1 - train_acc) 
            y_pred_test = model(X_test)
            loss_test = criterion(y_pred_test, y_test.t())
            losses_test.append(loss_test.data)
            pred_labels_test = torch.max(y_pred_test, 1, keepdim=True)[1]
            true_labels_test = torch.max(y_test.t(), 1, keepdim=True)[1]
            test_acc = true_labels_test.eq(pred_labels_test.data).sum().to(dtype=torch.float)/len(true_labels_test)
            errors_test.append(1 - test_acc)
            if errors_test[-1] <= final_error:
                stop_word = True
                break
            if N_iter >= iter_limit:
                N_iter = None
                return N_iter, thetas, losses_train,losses_test, errors_train, errors_test
        model.train()
        theta_t = make_splitting_step(theta_t.cpu(), Qs[i_batch].cpu(), Rs[i_batch].cpu(), y_trains[i_batch].cpu(), stepsize, n_train).to(dtype=torch.float)
        model = model_init(model, theta_t.t())
        N_iter += 1  
    return N_iter, thetas, losses_train,losses_test, errors_train, errors_test

# Drawing Helper
def drawLearningRateTime(learning_rates, list_of_methods, list_of_labels):
    colors = ['g', 'r']
    color_labels = ['^', 'o']
    plt.figure(figsize = (3.5,2.5))
    for method, label, color, col_lab in zip(list_of_methods, list_of_labels, colors, color_labels):
        mean = np.zeros(len(learning_rates))
        std = np.zeros(len(learning_rates))
        for i_lr, lr in enumerate(learning_rates):
            if any(method[:, i_lr]) == None:
                mean[i_lr] = None
                std[i_lr]  = None
            else:
                mean[i_lr] = np.mean(method[:, i_lr])
                std[i_lr]  = np.std(method[:, i_lr])
        plt.loglog(learning_rates, mean, color+col_lab, label = label)
        plt.loglog(learning_rates, mean, color+':')
        plt.fill_between(learning_rates, mean-std, mean+std, color=color, alpha=0.1)
        plt.grid(True,which="both", linestyle='--', linewidth=0.4)
        plt.xlabel('Learning rate')
        plt.ylabel('Time to converge')
        plt.legend()
        
    plt.tight_layout()
    plt.show()

# Drawing Helper
def drawLearningRateIterCount(learning_rates, list_of_methods, list_of_labels):
    colors = ['g', 'r']
    color_labels = ['^', 'o']
    plt.figure(figsize = (3.5,2.5))
    for method, label, color, col_lab in zip(list_of_methods, list_of_labels, colors, color_labels):
        mean = np.zeros(len(learning_rates))
        std = np.zeros(len(learning_rates))
        for i_lr, lr in enumerate(learning_rates):
            if any(method[:, i_lr]) == None:
                mean[i_lr] = None
                std[i_lr]  = None
            else:
                mean[i_lr] = np.mean(method[:, i_lr])
                std[i_lr]  = np.std(method[:, i_lr])
        std = np.std(method, axis = 0)   
        plt.loglog(learning_rates, mean, color+col_lab, label = label)
        plt.loglog(learning_rates, mean, color+':')
        plt.fill_between(learning_rates, mean-std, mean+std, color=color, alpha=0.1)
        plt.grid(True,which="both", linestyle='--', linewidth=0.4)
        plt.xlabel('Learning rate')
        plt.ylabel('Iterations to converge')
        plt.legend()
    plt.tight_layout()
    plt.show()
    
print('Functions defined Successfuly!!')

In [0]:
X_trains, y_trains, X_test, y_test, Qs, Rs = load_batched_data(batch_size=batch_size, qr_mode = True, number_of_classes=number_of_classes)
s_train, batch_size, p = X_trains.shape 
n_train, K, n_test = s_train*batch_size, *y_test.shape
model = load_model(X_test, y_test)
for i_exp in progress_bar(range(N_EXPERIMENTS)):
    print(f' EXPERIMENTS {i_exp+1}/ {N_EXPERIMENTS} ')
    init_bound = 1.0/math.sqrt(p)
    theta_0 = init_bound*torch.FloatTensor(p, K).uniform_(-1, 1)
    for i_lr, learning_rate in enumerate(LEARNING_RATES):
        stepsize = learning_rate*n_train/batch_size
        print(f'learning_rate {learning_rate}, stepsize {stepsize} ')
        start_time = time.time()
        N_iter, thetas, losses_train,losses_test, errors_train, errors_test = \
            spl_training(theta_0,  Qs, Rs, X_trains, y_trains,  X_test, y_test, stepsize, model, final_error = TARGET_ERROR, iter_limit=iter_limit)
        end_time = time.time()
        SPL_time[i_exp, i_lr] = end_time - start_time
        SPL_N_iter[i_exp, i_lr] = N_iter
        if N_iter == None:
            SPL_N_iter[i_exp, i_lr] = None
        start_time = time.time()
        N_iter, thetas, losses_train,losses_test, errors_train, errors_test = \
            sgd_training(theta_0, X_trains, y_trains,  X_test, y_test, learning_rate, model, final_error = TARGET_ERROR, iter_limit=iter_limit)
        end_time = time.time()
        SGD_time[i_exp, i_lr] = end_time - start_time
        SGD_N_iter[i_exp, i_lr] = N_iter
        if N_iter == None:
            SGD_time[i_exp, i_lr] = None
        drawLearningRateTime(LEARNING_RATES, [SPL_time, SGD_time], ['Splitting','SGD'])
        drawLearningRateIterCount(LEARNING_RATES, [SPL_N_iter, SGD_N_iter], ['Splitting','SGD'])